In [123]:
import pandas as pd
import numpy as np
'''
data preprocessing
'''

#주가데이터 파일을 로드
csv_data = pd.read_csv("./data/005930_200325.csv")
csv_data

,date,start,high,low,end,volume,foreign,person,company,wondollar
0,2020.3.25,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9
1,2020.3.24,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6
2,2020.3.23,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5
3,2020.3.20,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5
4,2020.3.19,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7
...,...,...,...,...,...,...,...,...,...,...
592,2017.10.23,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2
593,2017.10.20,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0
594,2017.10.19,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4
595,2017.10.18,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9


In [126]:
#필요한 부분을 가져온다
stdata = csv_data[["start","high", "low","end","volume","foreign", "person","company","wondollar"]]
stdata.head()

#전일대비 가격 변화율을 추가한다
price_change = -np.diff(stdata["end"].to_numpy())
price = stdata["end"].to_numpy()
price = price[1:]
price_change_rate = (price_change/price)*100
price_change_rate=np.append(price_change_rate,[0])
stdata['price_change_rate'] = price_change_rate


#캔들 (종가,시가,고가,저가) 관련 정보를 추가한다
#시가-종가
start = stdata["start"].to_numpy()
end = stdata["end"].to_numpy()
candle_end_start = end-start
stdata['candle_end_start'] = candle_end_start

#고가-저가 (당일변동폭) 
high = stdata["high"].to_numpy()
low = stdata["low"].to_numpy()
candle_high_low = high-low
stdata['candle_high_low'] = candle_high_low


#상승or하락 표시 상승:1 보함:0 하락:-1
up_or_down = np.zeros(597)

price_change_rate = stdata["price_change_rate"].to_numpy()
for idx, val in enumerate(price_change_rate):
    if val>0:
        up_or_down[idx]=1
    if val<0:
        up_or_down[idx]=-1
stdata['up_or_down'] = up_or_down

stdata


,start,high,low,end,volume,foreign,person,company,wondollar,price_change_rate,candle_end_start,candle_high_low,up_or_down
0,48950,49600,47150,48650,52735922,-2751527,4980190,-1695448,1229.9,3.620873,-300,2450,1.0
1,43850,46950,43050,46950,49801908,2929769,-6174614,3254324,1249.6,10.470588,3100,3900,1.0
2,42600,43550,42400,42500,41701626,-8143293,10988297,-3248991,1266.5,-6.387665,-100,1150,-1.0
3,44150,45500,43550,45400,49730008,-5133212,3248749,1669124,1246.5,5.704307,1250,1950,1.0
4,46400,46650,42300,42950,56925513,-5807616,4105594,1612440,1285.7,-5.811404,-3450,4350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,54600,54640,54000,54300,8311050,30468,4568,-63065,1130.2,0.854383,-300,640,1.0
593,52800,54100,52800,53840,8027050,29906,-2283,-39735,1131.0,1.623254,1040,1300,1.0
594,54700,54700,52980,52980,12108700,-19892,46171,-46270,1132.4,-3.250548,-1720,1720,-1.0
595,54820,55240,54040,54760,10110750,8951,-3285,-23873,1129.9,-0.072993,-60,1200,-1.0


In [112]:
#정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
fitted = scaler.fit(stdata)
output = scaler.transform(stdata)
output.shape

(597, 10)

In [110]:
#정답1 : 가격변화강도 = 가격 변화율 절대값
price_change = -np.diff(stdata["end"].to_numpy())

price = stdata["end"].to_numpy()
price = price[1:]

price_change_rate = (price_change/price)*100
price_change_power = abs(price_change_rate)

price_change_power

array([ 3.62087327, 10.47058824,  6.3876652 ,  5.70430733,  5.81140351,
        3.59408034,  3.27198364,  2.1021021 ,  1.67322835,  2.49520154,
        4.57875458,  0.73800738,  4.07079646,  2.24913495,  0.69686411,
        3.6101083 ,  0.72727273,  1.47601476,  3.0411449 ,  1.0619469 ,
        2.417962  ,  1.93661972,  4.05405405,  1.33333333,  0.33222591,
        0.66889632,  2.76422764,  0.48543689,  1.8121911 ,  0.33057851,
        1.00166945,  0.33500838,  1.1589404 ,  1.14566285,  2.68907563,
        1.01867572,  2.97202797,  1.41843972,  1.3986014 ,  3.21489002,
        0.51020408,  3.28947368,  2.40770465,  1.46579805,  1.6025641 ,
        1.79445351,  0.98846787,  2.88135593,  1.66666667,  0.        ,
        0.84033613,  1.53583618,  3.16901408,  1.7921147 ,  0.54054054,
        0.        ,  0.54347826,  1.07526882,  1.23893805,  1.98555957,
        0.72727273,  0.9009009 ,  0.89285714,  0.        ,  0.53285968,
        0.70546737,  3.65630713,  0.        ,  2.62664165,  2.69

In [93]:
#정답2 : 상승(변화율>0) or 하락 
#상승일경우 1 하락일경우 0

up_or_down = np.zeros(596)

price_change = -np.diff(stdata["end"].to_numpy())
for idx, val in enumerate(price_change):
    if val>0:
        up_or_down[idx]=1

up_or_down.shape

(596,)

In [52]:
output[1]

array([-0.88161839, -0.3533235 , -0.95326911, -0.25315563,  4.81360288,
        1.23418063, -3.02635004,  1.86121588,  2.50288087])